# Let's do the analysis with Keras

### Import packages

In [1]:
import numpy as np #necessary for keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, clone_model #sequential model, we define the layers in a sequential way
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import time
import math
from sklearn.model_selection import GridSearchCV
#from keras.wrappers.scikit_learn import KerasClassifier
#import keras,sklearn
from collections import deque

### Import the file we have made before

In [2]:
fname = 'DATA/sequences16.csv'
sx, sy = np.loadtxt(fname, delimiter=',', usecols =(0,1), unpack =True, dtype= str) #make 2 arrays of (10000,0) with x and y
N=len(sy)
print(N)
Ls = len(sx[0])
print(Ls)

print(sx[0],sy[0]) #let's check the first one

perc_train = 0.8
N_train = int(N*perc_train)
N_test = N - N_train

print(f'\ndata : {N}\ntrain: {N_train}\ntest: {N_test}')

3000
16
AAGGTCTGCCGGCCGA 1

data : 3000
train: 2400
test: 600


In [3]:
Q = ['A','C','G','T']
Nc = 4
onehc= {Q[i]:i for i in range(Nc)} #dictionary used to convert everything in binary
print(onehc) #1000 0100 0010 0001

{'A': 0, 'C': 1, 'G': 2, 'T': 3}


### DATA conversion


In [4]:
y = sy.astype(int)

L = Ls*Nc
print(L) #lenght for the array we are going to use
x = np.zeros((N,L))
print(x[0])

for n in range(N):
    for i in range(Ls): #for every character
        x[n][i*4 + onehc[sx[n][i]]] = 1 #convert the letter in binary
        
print(sx[0]) #check
print(x[0]) #check

64
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
AAGGTCTGCCGGCCGA
[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]


### Splitting train and test



In [5]:
(x_train, y_train) = (x[:N_train],y[:N_train])
(x_test, y_test) = (x[N_train:],y[N_train:])

#y_train=np.array([y_train]).T

print(x_train.shape)
print(y_train.shape)

print(y_train.sum() / N_train)
print(y_test.sum()/N_test)

(2400, 64)
(2400,)
0.24333333333333335
0.25833333333333336


### NON COMPILARE

In [ ]:
np.random.seed(123)

model = Sequential()

#input layer matches the size of input layer in keras
model.add(Dense(L,input_shape = (L,),activation = 'relu')) #the first one has the size of the input
model.add(Dense(L/2,activation = 'relu')) #l/2 size
model.add(Dense(L/4,activation = 'relu')) # we can put another activation
model.add(Dropout(0.2)) #only in the previous layer
model.add(Dense(1,activation = 'sigmoid')) #output layer

print(model.summary())

In [ ]:
model.compile(loss = 'binary_crossentropy', #1 or 0 with log scale
              optimizer = 'adam',
              metrics = ['accuracy']) 

In [ ]:
fit = model.fit(x_train, y_train,
               epochs = 70, batch_size = 50,
               validation_data = (x_test,y_test),
               shuffle = True) #after one epochs it shuffle the order of data

In [ ]:
for obs in ('accuracy', 'loss'):
    plt.figure(figsize = (6,4))
    plt.plot(fit.history[obs], 'r', label = obs +' of training data')
    
    plt.plot(fit.history['val_' + obs], 'b--', label = obs + ' of validation data')
    plt.title('Adam')
    plt.ylabel(obs)
    plt.xlabel('epochs')
    plt.legend()
    plt.show()

In [ ]:
np.random.seed(123)

model1 = Sequential()

#input layer matches the size of input layer in keras
model1.add(Dense(L,input_shape = (L,),activation = 'relu')) #the first one has the size of the input
model1.add(Dense(L/2,activation = 'relu')) #l/2 size
model1.add(Dense(L/4,activation = 'relu')) # we can put another activation
model1.add(Dropout(0.2)) #only in the previous layer
model1.add(Dense(1,activation = 'sigmoid')) #output layer

print(model1.summary())

In [ ]:
opt1=SGD(learning_rate=0.01,
        momentum = 0.9, #if = 0 is vanilla
        nesterov = True)

model1.compile(loss = 'binary_crossentropy', #1 or 0 with log scale
              optimizer = opt1,
              metrics = ['accuracy']) 

In [ ]:
fit1 = model1.fit(x_train, y_train,
               epochs = 70, batch_size = 50,
               validation_data = (x_test,y_test),
               shuffle = True) #after one epochs it shuffle the order of data

In [ ]:
for obs in ('accuracy', 'loss'):
    plt.figure(figsize = (6,4))
    plt.plot(fit1.history[obs], 'r', label = obs +' of training data')
    
    plt.plot(fit1.history['val_' + obs], 'b--', label = obs + ' of validation data')
    plt.title('sgd')
    plt.ylabel(obs)
    plt.xlabel('epochs')
    plt.legend()
    plt.show()

In [ ]:
# def create_DNN(activation='relu'):
#     # instantiate model
#     model = Sequential()
#     # add a dense all-to-all relu layer 
#     model.add(Dense(L,input_shape = (L,),activation = activation)) #the first one has the size of the input
#     model.add(Dense(L/2,activation = activation)) #l/2 size
#     model.add(Dense(L/4,activation = activation)) # we can put another activation
#     model.add(Dropout(0.2)) #only in the previous layer
#     model.add(Dense(1,activation = 'sigmoid')) #output layer

#     return model

In [ ]:
# def compile_model(optimizer=keras.optimizers.Adam()):
#     # create the mode
#     model=create_DNN()
#     # compile the model
#     model.compile(loss='binary_crossentropy',
#                   optimizer=optimizer,
#                   metrics=['accuracy'])
#     return model

### COMINCIARE A COMPILARE

Function used to create the model.

In [7]:
def create_model(activation='relu',optimizer='adam',dropout_rate=0.2):
# create model
    model = Sequential()
    model.add(Dense(L,input_shape = (L,),activation = activation)) #the first one has the size of the input
    model.add(Dense(L/2,activation = activation)) #l/2 size
    model.add(Dense(L/4,activation = activation)) # we can put another activation
    model.add(Dropout(dropout_rate)) #only in the previous layer
    model.add(Dense(1,activation = 'sigmoid')) #output layer
# compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

GridsearchCV on epochs and batch_size.

In [9]:
#gridseach on epochs and batch size

model_gridsearch = KerasClassifier(build_fn=create_model, 
                        verbose=1)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model_gridsearch, param_grid=param_grid, n_jobs=1, cv=4)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/10
180/180 [==============================] - 1s 2ms/step - loss: 0.6040 - accuracy: 0.6865
Epoch 2/10
180/180 [==============================] - 0s 1ms/step - loss: 0.5238 - accuracy: 0.7505
Epoch 3/10
180/180 [==============================] - 0s 1ms/step - loss: 0.4635 - accuracy: 0.7700
Epoch 4/10
180/180 [==============================] - 0s 1ms/step - loss: 0.4384 - accuracy: 0.7979
Epoch 5/10
180/180 [==============================] - 0s 1ms/step - loss: 0.4230 - accuracy: 0.7958
Epoch 6/10
180/180 [==============================] - 0s 1ms/step - loss: 0.3592 - accuracy: 0.8599
Epoch 7/10
180/180 [==============================] - 0s 1ms/step - loss: 0.2935 - accuracy: 0.8738
Epoch 8/10
180/180 [==============================] - 0s 1ms/step - loss: 0.2476 - accuracy: 0.9052
Epoch 9/10
180/180 [==============================] - 0s 1ms/step - loss: 0.1778 - accuracy: 0.9424
Epoch 10/10
60/60 [==============================] - 0s 2ms/step - loss: 0.5141 - accuracy: 0.8150
E

180/180 [==============================] - 0s 1ms/step - loss: 0.0221 - accuracy: 0.9952
Epoch 17/50
180/180 [==============================] - 0s 1ms/step - loss: 0.0152 - accuracy: 0.9959
Epoch 18/50
180/180 [==============================] - 0s 2ms/step - loss: 0.0115 - accuracy: 0.9984
Epoch 19/50
180/180 [==============================] - 0s 1ms/step - loss: 0.0083 - accuracy: 0.9993
Epoch 20/50
180/180 [==============================] - 0s 2ms/step - loss: 0.0156 - accuracy: 0.9968
Epoch 21/50
180/180 [==============================] - 0s 1ms/step - loss: 0.0075 - accuracy: 0.9990
Epoch 22/50
180/180 [==============================] - 0s 1ms/step - loss: 0.0058 - accuracy: 0.9997
Epoch 23/50
180/180 [==============================] - 0s 1ms/step - loss: 0.0271 - accuracy: 0.9902
Epoch 24/50
180/180 [==============================] - 0s 1ms/step - loss: 0.0514 - accuracy: 0.9834
Epoch 25/50
180/180 [==============================] - 0s 1ms/step - loss: 0.0153 - accuracy: 0.9968
Ep

180/180 [==============================] - 0s 1ms/step - loss: 2.4447e-04 - accuracy: 1.0000
Epoch 74/100
180/180 [==============================] - 0s 2ms/step - loss: 1.5433e-04 - accuracy: 1.0000
Epoch 75/100
180/180 [==============================] - 0s 1ms/step - loss: 3.0401e-04 - accuracy: 1.0000
Epoch 76/100
180/180 [==============================] - 0s 1ms/step - loss: 9.8303e-05 - accuracy: 1.0000
Epoch 77/100
180/180 [==============================] - 0s 1ms/step - loss: 8.8001e-05 - accuracy: 1.0000
Epoch 78/100
180/180 [==============================] - 0s 1ms/step - loss: 7.8856e-05 - accuracy: 1.0000
Epoch 79/100
180/180 [==============================] - 0s 2ms/step - loss: 7.1568e-05 - accuracy: 1.0000
Epoch 80/100
180/180 [==============================] - 0s 2ms/step - loss: 1.7330e-04 - accuracy: 1.0000
Epoch 81/100
180/180 [==============================] - 0s 1ms/step - loss: 3.8284e-04 - accuracy: 1.0000
Epoch 82/100
180/180 [==============================] - 0s 

180/180 [==============================] - 0s 1ms/step - loss: 0.0296 - accuracy: 0.9934
Epoch 30/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0084 - accuracy: 0.9973
Epoch 31/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0065 - accuracy: 0.9997
Epoch 32/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 0.9990
Epoch 33/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 34/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 35/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 36/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 37/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0153 - accuracy: 0.9949
Epoch 38/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0558 - accuracy: 

180/180 [==============================] - 0s 1ms/step - loss: 0.0061 - accuracy: 0.9996
Epoch 87/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0055 - accuracy: 0.9980
Epoch 88/100
180/180 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 89/100
180/180 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 90/100
180/180 [==============================] - 0s 2ms/step - loss: 7.8770e-04 - accuracy: 1.0000
Epoch 91/100
180/180 [==============================] - 0s 2ms/step - loss: 7.5456e-04 - accuracy: 1.0000
Epoch 92/100
180/180 [==============================] - 0s 2ms/step - loss: 5.4352e-04 - accuracy: 1.0000
Epoch 93/100
180/180 [==============================] - 0s 2ms/step - loss: 4.6569e-04 - accuracy: 1.0000
Epoch 94/100
180/180 [==============================] - 0s 2ms/step - loss: 5.8287e-04 - accuracy: 1.0000
Epoch 95/100
180/180 [==============================] - 0s 2ms/step - loss:

...

18/18 [==============================] - 0s 1ms/step - loss: 0.3807 - accuracy: 0.8351
Epoch 14/50
18/18 [==============================] - 0s 2ms/step - loss: 0.3703 - accuracy: 0.8403
Epoch 15/50
18/18 [==============================] - 0s 1ms/step - loss: 0.3389 - accuracy: 0.8615
Epoch 16/50
18/18 [==============================] - 0s 2ms/step - loss: 0.3377 - accuracy: 0.8675
Epoch 17/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2930 - accuracy: 0.8876
Epoch 18/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2799 - accuracy: 0.8841
Epoch 19/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2579 - accuracy: 0.9070
Epoch 20/50
18/18 [==============================] - 0s 1ms/step - loss: 0.2363 - accuracy: 0.9241
Epoch 21/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2083 - accuracy: 0.9335
Epoch 22/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2030 - accuracy: 0.9304
Epoch 23/50
18/18 [===

18/18 [==============================] - 0s 2ms/step - loss: 0.0169 - accuracy: 0.9973
Epoch 46/50
18/18 [==============================] - 0s 2ms/step - loss: 0.0106 - accuracy: 0.9992
Epoch 47/50
18/18 [==============================] - 0s 2ms/step - loss: 0.0122 - accuracy: 0.9991
Epoch 48/50
18/18 [==============================] - 0s 2ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 49/50
18/18 [==============================] - 0s 2ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 50/50
6/6 [==============================] - 0s 2ms/step - loss: 0.9459 - accuracy: 0.7833
Epoch 1/100
18/18 [==============================] - 1s 2ms/step - loss: 0.6724 - accuracy: 0.5722
Epoch 2/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5720 - accuracy: 0.7504
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5711 - accuracy: 0.7526
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5528 - accuracy: 0.7513
Epoch 5/100
18/18 [=====

18/18 [==============================] - 0s 2ms/step - loss: 0.0067 - accuracy: 0.9998
Epoch 57/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 58/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 59/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 60/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 61/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 62/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 63/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 64/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 65/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 66/100


18/18 [==============================] - 0s 2ms/step - loss: 0.3764 - accuracy: 0.8453
Epoch 18/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3556 - accuracy: 0.8505
Epoch 19/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3419 - accuracy: 0.8474
Epoch 20/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3136 - accuracy: 0.8611
Epoch 21/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3037 - accuracy: 0.8665
Epoch 22/100
18/18 [==============================] - 0s 2ms/step - loss: 0.2549 - accuracy: 0.9055
Epoch 23/100
18/18 [==============================] - 0s 2ms/step - loss: 0.2435 - accuracy: 0.9060
Epoch 24/100
18/18 [==============================] - 0s 2ms/step - loss: 0.2033 - accuracy: 0.9370
Epoch 25/100
18/18 [==============================] - 0s 2ms/step - loss: 0.2091 - accuracy: 0.9272
Epoch 26/100
18/18 [==============================] - 0s 2ms/step - loss: 0.1657 - accuracy: 0.9617
Epoch 27/100


40/40 [==============================] - 0s 2ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 80/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 81/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 82/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 83/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 84/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 85/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 86/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 0.9998
Epoch 87/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0018 - accuracy: 0.9997
Epoch 88/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 89/100


In [13]:
batch_size = grid_result.best_params_['batch_size']
epochs = grid_result.best_params_['epochs']

GridsearchCV on activation functions.

In [15]:
#activation funcion
model_gridsearch = KerasClassifier(build_fn=create_model, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        verbose=1)

activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)

grid = GridSearchCV(estimator=model_gridsearch, param_grid=param_grid, n_jobs=1, cv=4)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
36/36 [==============================] - 1s 3ms/step - loss: 0.6789 - accuracy: 0.7166
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6560 - accuracy: 0.7636
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6406 - accuracy: 0.7542
Epoch 4/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6295 - accuracy: 0.7370
Epoch 5/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6154 - accuracy: 0.7465
Epoch 6/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6057 - accuracy: 0.7498
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5963 - accuracy: 0.7455
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5906 - accuracy: 0.7441
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5753 - accuracy: 0.7643
Epoch 10/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5708 - accuracy: 0.7637
Epoch 11/50
36/36 [

36/36 [==============================] - 0s 2ms/step - loss: 0.5439 - accuracy: 0.7489
Epoch 34/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5372 - accuracy: 0.7462
Epoch 35/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5196 - accuracy: 0.7627
Epoch 36/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5191 - accuracy: 0.7587
Epoch 37/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5164 - accuracy: 0.7559
Epoch 38/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5022 - accuracy: 0.7587
Epoch 39/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4937 - accuracy: 0.7671
Epoch 40/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4960 - accuracy: 0.7714
Epoch 41/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4784 - accuracy: 0.7711
Epoch 42/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4808 - accuracy: 0.7664
Epoch 43/50
36/36 [===

36/36 [==============================] - 0s 2ms/step - loss: 0.3969 - accuracy: 0.8161
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3967 - accuracy: 0.8008
Epoch 48/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3827 - accuracy: 0.8198
Epoch 49/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4014 - accuracy: 0.7935
Epoch 50/50
12/12 [==============================] - 0s 3ms/step - loss: 0.4956 - accuracy: 0.7750
Epoch 1/50
36/36 [==============================] - 1s 3ms/step - loss: 0.6002 - accuracy: 0.7024
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5662 - accuracy: 0.7582
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5480 - accuracy: 0.7577
Epoch 4/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5264 - accuracy: 0.7561
Epoch 5/50
36/36 [==============================] - 0s 3ms/step - loss: 0.5051 - accuracy: 0.7594
Epoch 6/50
36/36 [=========

36/36 [==============================] - 0s 3ms/step - loss: 0.4967 - accuracy: 0.7643
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4368 - accuracy: 0.8045
Epoch 10/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4401 - accuracy: 0.8079
Epoch 11/50
36/36 [==============================] - 0s 3ms/step - loss: 0.4464 - accuracy: 0.7789
Epoch 12/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4437 - accuracy: 0.8141
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4235 - accuracy: 0.8187
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4001 - accuracy: 0.8161
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4395 - accuracy: 0.7921
Epoch 16/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4225 - accuracy: 0.8111
Epoch 17/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4051 - accuracy: 0.8198
Epoch 18/50
36/36 [====

36/36 [==============================] - 0s 1ms/step - loss: 0.0552 - accuracy: 0.9941
Epoch 41/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0523 - accuracy: 0.9932
Epoch 42/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0512 - accuracy: 0.9891
Epoch 43/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0414 - accuracy: 0.9967
Epoch 44/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0287 - accuracy: 0.9982
Epoch 45/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0288 - accuracy: 0.9977
Epoch 46/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0243 - accuracy: 0.9998
Epoch 47/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0206 - accuracy: 1.0000
Epoch 48/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0176 - accuracy: 1.0000
Epoch 49/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0170 - accuracy: 1.0000
Epoch 50/50
12/12 [===

...

36/36 [==============================] - 0s 1ms/step - loss: 0.3397 - accuracy: 0.8449
Epoch 26/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3417 - accuracy: 0.8477
Epoch 27/50
36/36 [==============================] - 0s 1ms/step - loss: 0.3250 - accuracy: 0.8601
Epoch 28/50
36/36 [==============================] - 0s 1ms/step - loss: 0.3122 - accuracy: 0.8664
Epoch 29/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2821 - accuracy: 0.8842
Epoch 30/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2725 - accuracy: 0.9036
Epoch 31/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2569 - accuracy: 0.9043
Epoch 32/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2411 - accuracy: 0.9043
Epoch 33/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2402 - accuracy: 0.9161
Epoch 34/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2172 - accuracy: 0.9236
Epoch 35/50
36/36 [===

36/36 [==============================] - 0s 2ms/step - loss: 0.4518 - accuracy: 0.7731
Epoch 39/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4524 - accuracy: 0.7787
Epoch 40/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4667 - accuracy: 0.7658
Epoch 41/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4809 - accuracy: 0.7688
Epoch 42/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4486 - accuracy: 0.7941
Epoch 43/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4641 - accuracy: 0.7900
Epoch 44/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4581 - accuracy: 0.7808
Epoch 45/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4858 - accuracy: 0.7659
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4654 - accuracy: 0.7835
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4625 - accuracy: 0.7734
Epoch 48/50
36/36 [===

12/12 [==============================] - 0s 2ms/step - loss: 0.5465 - accuracy: 0.7267
Epoch 1/50
36/36 [==============================] - 2s 3ms/step - loss: 0.5833 - accuracy: 0.7478
Epoch 2/50
36/36 [==============================] - 0s 3ms/step - loss: 0.5714 - accuracy: 0.7496
Epoch 3/50
36/36 [==============================] - 0s 3ms/step - loss: 0.5573 - accuracy: 0.7568
Epoch 4/50
36/36 [==============================] - 0s 3ms/step - loss: 0.5574 - accuracy: 0.7551
Epoch 5/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5315 - accuracy: 0.7732
Epoch 6/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5489 - accuracy: 0.7554
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5483 - accuracy: 0.7561
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5535 - accuracy: 0.7408
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5483 - accuracy: 0.7450
Epoch 10/50
36/36 [============

36/36 [==============================] - 0s 2ms/step - loss: 0.4306 - accuracy: 0.8139
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4546 - accuracy: 0.7876
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4804 - accuracy: 0.7791
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.7775
Epoch 16/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4661 - accuracy: 0.7924
Epoch 17/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4598 - accuracy: 0.7895
Epoch 18/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4660 - accuracy: 0.7870
Epoch 19/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4513 - accuracy: 0.7894
Epoch 20/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4468 - accuracy: 0.8014
Epoch 21/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4426 - accuracy: 0.7957
Epoch 22/50
36/36 [===

36/36 [==============================] - 0s 2ms/step - loss: 0.4709 - accuracy: 0.7803
Epoch 45/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4747 - accuracy: 0.7724
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4650 - accuracy: 0.7818
Epoch 47/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4798 - accuracy: 0.7724
Epoch 48/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4868 - accuracy: 0.7605
Epoch 49/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4682 - accuracy: 0.7736
Epoch 50/50
12/12 [==============================] - 0s 1ms/step - loss: 0.5054 - accuracy: 0.7617
Epoch 1/50
36/36 [==============================] - 1s 2ms/step - loss: 0.6601 - accuracy: 0.6375
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5367 - accuracy: 0.7583
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5015 - accuracy: 0.7637
Epoch 4/50
36/36 [=======

GridsearchCV on optimizers.

In [13]:
# optimizers
model_gridsearch = KerasClassifier(build_fn=create_model, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        verbose=1)


optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

grid = GridSearchCV(estimator=model_gridsearch, param_grid=param_grid, n_jobs=1, cv=4)
grid_result = grid.fit(x_train,y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
36/36 [==============================] - 1s 3ms/step - loss: 0.6871 - accuracy: 0.5460
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6310 - accuracy: 0.7176
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5844 - accuracy: 0.7648
Epoch 4/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5861 - accuracy: 0.7382
Epoch 5/50
36/36 [==============================] - 0s 3ms/step - loss: 0.5722 - accuracy: 0.7492
Epoch 6/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5589 - accuracy: 0.7514
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5759 - accuracy: 0.7326
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5608 - accuracy: 0.7531
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5404 - accuracy: 0.7676
Epoch 10/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5446 - accuracy: 0.7609
Epoch 11/50
36/36 [

36/36 [==============================] - 0s 1ms/step - loss: 0.4948 - accuracy: 0.7701
Epoch 34/50
36/36 [==============================] - 0s 1ms/step - loss: 0.5147 - accuracy: 0.7499
Epoch 35/50
36/36 [==============================] - 0s 1ms/step - loss: 0.5089 - accuracy: 0.7630
Epoch 36/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4989 - accuracy: 0.7643
Epoch 37/50
36/36 [==============================] - 0s 1ms/step - loss: 0.5069 - accuracy: 0.7578
Epoch 38/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4828 - accuracy: 0.7737
Epoch 39/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4888 - accuracy: 0.7675
Epoch 40/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4979 - accuracy: 0.7606
Epoch 41/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4867 - accuracy: 0.7626
Epoch 42/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4873 - accuracy: 0.7661
Epoch 43/50
36/36 [===

36/36 [==============================] - 0s 2ms/step - loss: 0.0076 - accuracy: 0.9991
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0051 - accuracy: 0.9999
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 48/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 49/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0036 - accuracy: 0.9998
Epoch 50/50
12/12 [==============================] - 0s 2ms/step - loss: 1.6119 - accuracy: 0.7817
Epoch 1/50
36/36 [==============================] - 1s 2ms/step - loss: 0.5694 - accuracy: 0.7454
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5051 - accuracy: 0.7598
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4643 - accuracy: 0.7674
Epoch 4/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4502 - accuracy: 0.7704
Epoch 5/50
36/36 [========

36/36 [==============================] - 0s 2ms/step - loss: 0.6191 - accuracy: 0.7240
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5968 - accuracy: 0.7565
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6083 - accuracy: 0.7245
Epoch 10/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5948 - accuracy: 0.7424
Epoch 11/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5941 - accuracy: 0.7394
Epoch 12/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5788 - accuracy: 0.7554
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5784 - accuracy: 0.7479
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5801 - accuracy: 0.7481
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5869 - accuracy: 0.7347
Epoch 16/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5735 - accuracy: 0.7510
Epoch 17/50
36/36 [=====

36/36 [==============================] - 0s 2ms/step - loss: 0.5468 - accuracy: 0.7616
Epoch 40/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5399 - accuracy: 0.7574
Epoch 41/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5555 - accuracy: 0.7529
Epoch 42/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5397 - accuracy: 0.7623
Epoch 43/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5389 - accuracy: 0.7630
Epoch 44/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5594 - accuracy: 0.7484
Epoch 45/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5409 - accuracy: 0.7564
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5372 - accuracy: 0.7682
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5352 - accuracy: 0.7705
Epoch 48/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5351 - accuracy: 0.7570
Epoch 49/50
36/36 [===

...

36/36 [==============================] - 0s 2ms/step - loss: 0.2828 - accuracy: 0.8881
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2527 - accuracy: 0.9165
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2246 - accuracy: 0.9326
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2054 - accuracy: 0.9404
Epoch 16/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1725 - accuracy: 0.9493
Epoch 17/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1488 - accuracy: 0.9557
Epoch 18/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1306 - accuracy: 0.9678
Epoch 19/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1020 - accuracy: 0.9764
Epoch 20/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0957 - accuracy: 0.9776
Epoch 21/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0674 - accuracy: 0.9905
Epoch 22/50
36/36 [===

36/36 [==============================] - 0s 2ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 45/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0044 - accuracy: 0.9996
Epoch 48/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0080 - accuracy: 0.9990
Epoch 49/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 0.9985
Epoch 50/50
12/12 [==============================] - 0s 1ms/step - loss: 1.2603 - accuracy: 0.7733
Epoch 1/50
36/36 [==============================] - 1s 2ms/step - loss: 0.6784 - accuracy: 0.5476
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5623 - accuracy: 0.7606
Epoch 3/50
36/36 [==============================] - 0s 1ms/step - loss: 0.5383 - accuracy: 0.7518
Epoch 4/50
36/36 [=======

36/36 [==============================] - 0s 2ms/step - loss: 0.5098 - accuracy: 0.7527
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5037 - accuracy: 0.7540
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4701 - accuracy: 0.7697
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4505 - accuracy: 0.7847
Epoch 10/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4672 - accuracy: 0.7755
Epoch 11/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4319 - accuracy: 0.8038
Epoch 12/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4468 - accuracy: 0.7910
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4549 - accuracy: 0.7823
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4191 - accuracy: 0.8083
Epoch 15/50
36/36 [==============================] - 0s 3ms/step - loss: 0.4171 - accuracy: 0.8064
Epoch 16/50
36/36 [======

36/36 [==============================] - 0s 2ms/step - loss: 0.2684 - accuracy: 0.9015
Epoch 39/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2627 - accuracy: 0.9049
Epoch 40/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2552 - accuracy: 0.9131
Epoch 41/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2373 - accuracy: 0.9107
Epoch 42/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2171 - accuracy: 0.9296
Epoch 43/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2309 - accuracy: 0.9265
Epoch 44/50
36/36 [==============================] - 0s 1ms/step - loss: 0.2201 - accuracy: 0.9237
Epoch 45/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2064 - accuracy: 0.9261
Epoch 46/50
36/36 [==============================] - 0s 1ms/step - loss: 0.1989 - accuracy: 0.9325
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2019 - accuracy: 0.9261
Epoch 48/50
36/36 [===

36/36 [==============================] - 0s 1ms/step - loss: 0.0073 - accuracy: 0.9983
Epoch 50/50
12/12 [==============================] - 0s 1ms/step - loss: 1.2666 - accuracy: 0.7583
Epoch 1/50
36/36 [==============================] - 2s 3ms/step - loss: 0.6403 - accuracy: 0.6537
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5462 - accuracy: 0.7499
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5032 - accuracy: 0.7611
Epoch 4/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5090 - accuracy: 0.7496
Epoch 5/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4584 - accuracy: 0.7784
Epoch 6/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4448 - accuracy: 0.7807
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4398 - accuracy: 0.7980
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4165 - accuracy: 0.8290
Epoch 9/50
36/36 [============

GridsearchCV on dropout_rate.

In [14]:
#dropout
model_gridsearch = KerasClassifier(build_fn=create_model, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        verbose=1)

dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate)

grid = GridSearchCV(estimator=model_gridsearch, param_grid=param_grid, n_jobs=1, cv=4)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
36/36 [==============================] - 1s 3ms/step - loss: 0.5985 - accuracy: 0.7306
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5262 - accuracy: 0.7446
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4800 - accuracy: 0.7646
Epoch 4/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4636 - accuracy: 0.7747
Epoch 5/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4220 - accuracy: 0.8096
Epoch 6/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4125 - accuracy: 0.8053
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4075 - accuracy: 0.8147
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3795 - accuracy: 0.8349
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3482 - accuracy: 0.8422
Epoch 10/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3199 - accuracy: 0.8759
Epoch 11/50
36/36 [

36/36 [==============================] - 0s 2ms/step - loss: 0.2914 - accuracy: 0.8864
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2514 - accuracy: 0.9116
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2257 - accuracy: 0.9224
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1875 - accuracy: 0.9369
Epoch 16/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1601 - accuracy: 0.9521
Epoch 17/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1531 - accuracy: 0.9603
Epoch 18/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0984 - accuracy: 0.9778
Epoch 19/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0778 - accuracy: 0.9899
Epoch 20/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0642 - accuracy: 0.9914
Epoch 21/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0460 - accuracy: 0.9981
Epoch 22/50
36/36 [===

36/36 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 45/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 47/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 48/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 49/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 50/50
12/12 [==============================] - 0s 1ms/step - loss: 1.0376 - accuracy: 0.8217
Epoch 1/50
36/36 [==============================] - 1s 2ms/step - loss: 0.6103 - accuracy: 0.7068
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5231 - accuracy: 0.7618
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4869 - accuracy: 0.7704
Epoch 4/50
36/36 [=======

36/36 [==============================] - 0s 2ms/step - loss: 0.4692 - accuracy: 0.7614
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4251 - accuracy: 0.7948
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4105 - accuracy: 0.8076
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3823 - accuracy: 0.8206
Epoch 10/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3522 - accuracy: 0.8412
Epoch 11/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3210 - accuracy: 0.8622
Epoch 12/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2873 - accuracy: 0.8942
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2460 - accuracy: 0.9076
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2309 - accuracy: 0.9211
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1822 - accuracy: 0.9377
Epoch 16/50
36/36 [======

36/36 [==============================] - 0s 2ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 39/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 40/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 41/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 42/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 43/50
36/36 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 44/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 45/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 48/50
36/36 [===

...

36/36 [==============================] - 0s 2ms/step - loss: 0.4186 - accuracy: 0.8064
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4069 - accuracy: 0.7998
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3712 - accuracy: 0.8365
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3435 - accuracy: 0.8446
Epoch 16/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3451 - accuracy: 0.8332
Epoch 17/50
36/36 [==============================] - 0s 2ms/step - loss: 0.3239 - accuracy: 0.8703
Epoch 18/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2932 - accuracy: 0.8732
Epoch 19/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2671 - accuracy: 0.9000
Epoch 20/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2541 - accuracy: 0.9034
Epoch 21/50
36/36 [==============================] - 0s 2ms/step - loss: 0.2326 - accuracy: 0.9069
Epoch 22/50
36/36 [===

36/36 [==============================] - 0s 2ms/step - loss: 0.1224 - accuracy: 0.9129
Epoch 45/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1270 - accuracy: 0.9106
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1255 - accuracy: 0.9117
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1207 - accuracy: 0.9202
Epoch 48/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1018 - accuracy: 0.9353
Epoch 49/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0957 - accuracy: 0.9444
Epoch 50/50
12/12 [==============================] - 0s 1ms/step - loss: 1.1384 - accuracy: 0.7833
Epoch 1/50
36/36 [==============================] - 1s 3ms/step - loss: 0.7100 - accuracy: 0.5975
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5893 - accuracy: 0.7572
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5646 - accuracy: 0.7639
Epoch 4/50
36/36 [=======

36/36 [==============================] - 0s 2ms/step - loss: 0.5773 - accuracy: 0.7615
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5666 - accuracy: 0.7458
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5584 - accuracy: 0.7519
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5378 - accuracy: 0.7707
Epoch 10/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5181 - accuracy: 0.7824
Epoch 11/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5387 - accuracy: 0.7728
Epoch 12/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5160 - accuracy: 0.7818
Epoch 13/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5071 - accuracy: 0.7645
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5065 - accuracy: 0.7825
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4998 - accuracy: 0.7762
Epoch 16/50
36/36 [======

36/36 [==============================] - 0s 2ms/step - loss: 0.1491 - accuracy: 0.9614
Epoch 39/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1370 - accuracy: 0.9666
Epoch 40/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1540 - accuracy: 0.9553
Epoch 41/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1427 - accuracy: 0.9722
Epoch 42/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1304 - accuracy: 0.9726
Epoch 43/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1388 - accuracy: 0.9661
Epoch 44/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1407 - accuracy: 0.9616
Epoch 45/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1288 - accuracy: 0.9689
Epoch 46/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1165 - accuracy: 0.9712
Epoch 47/50
36/36 [==============================] - 0s 2ms/step - loss: 0.1274 - accuracy: 0.9683
Epoch 48/50
36/36 [===

12/12 [==============================] - 0s 2ms/step - loss: 0.5560 - accuracy: 0.7917
Epoch 1/50
36/36 [==============================] - 1s 2ms/step - loss: 0.6836 - accuracy: 0.7260
Epoch 2/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6238 - accuracy: 0.7650
Epoch 3/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6429 - accuracy: 0.7464
Epoch 4/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6079 - accuracy: 0.7459
Epoch 5/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6112 - accuracy: 0.7570
Epoch 6/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5981 - accuracy: 0.7568
Epoch 7/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5816 - accuracy: 0.7666
Epoch 8/50
36/36 [==============================] - 0s 2ms/step - loss: 0.6096 - accuracy: 0.7393
Epoch 9/50
36/36 [==============================] - 0s 2ms/step - loss: 0.5881 - accuracy: 0.7647
Epoch 10/50
36/36 [============

### Augmentation

In [9]:
def Augmentation2(s,y):
    Ls = int(len(s)/4)
    M = np.zeros((Ls-1,Ls*4))
    a = deque(s.tolist())
    a.rotate(4)
    M[0] = a
    for i in range(1,Ls-1):
        a = deque(M[i-1].tolist())
        a.rotate(4)
        M[i] = a
    return M, np.full(Ls-1,y)

In [10]:
# We stock the data augmentated in another array, so we can do some analysis
x_trainaug = np.zeros((N_train,L))
y_trainaug = np.zeros(N_train)
x_trainaug[:] = x_train[:]
y_trainaug[:] = y_train[:]

print(x_trainaug.shape)

for i in range(N_train):
    f = Augmentation2(x_train[i],y_train[i])
    x_trainaug = np.append(x_trainaug,f[0], axis=0)
    y_trainaug = np.append(y_trainaug,f[1]) 
    
print(x_trainaug.shape)

(2400, 64)
(38400, 64)


In [14]:
#activation funcion
model_gridsearch = KerasClassifier(build_fn=create_model, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        verbose=1)

activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model_gridsearch, param_grid=param_grid, n_jobs=1, cv=4)
grid_result = grid.fit(x_trainaug, y_trainaug)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/5
576/576 [==============================] - 2s 2ms/step - loss: 0.6176 - accuracy: 0.7538
Epoch 2/5
576/576 [==============================] - 1s 2ms/step - loss: 0.5623 - accuracy: 0.7512
Epoch 3/5
576/576 [==============================] - 1s 2ms/step - loss: 0.5522 - accuracy: 0.7477
Epoch 4/5
576/576 [==============================] - 1s 2ms/step - loss: 0.5186 - accuracy: 0.7540
Epoch 5/5
192/192 [==============================] - 0s 2ms/step - loss: 0.4741 - accuracy: 0.7720
Epoch 1/5
576/576 [==============================] - 2s 2ms/step - loss: 0.6048 - accuracy: 0.7628
Epoch 2/5
576/576 [==============================] - 1s 3ms/step - loss: 0.5504 - accuracy: 0.7617
Epoch 3/5
576/576 [==============================] - 1s 2ms/step - loss: 0.5327 - accuracy: 0.7614
Epoch 4/5
576/576 [==============================] - 1s 2ms/step - loss: 0.5021 - accuracy: 0.7606
Epoch 5/5
192/192 [==============================] - 0s 1ms/step - loss: 0.5400 - accuracy: 0.7453
Epoch 1/5


KeyboardInterrupt: 

In [ ]:
# call Keras scikit wrapper
model_gridsearch = KerasClassifier(build_fn=create_model, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        verbose=1)

# list of allowed optional arguments for the optimizer, see `compile_model()`
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# define parameter dictionary
param_grid = dict(optimizer=optimizer)
# call scikit grid search module
grid = GridSearchCV(estimator=model_gridsearch, param_grid=param_grid, n_jobs=1, cv=4)
grid_result = grid.fit(x_trainaug,y_trainaug)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
model_gridsearch = KerasClassifier(build_fn=create_model, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        verbose=1)
# define the grid search parameters
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model_gridsearch, param_grid=param_grid, n_jobs=1, cv=4)
grid_result = grid.fit(x_trainaug, y_trainaug)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))